In [2]:
# импорты и класс который даёт учебник

import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [3]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.dropna().describe()
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


список простых функций агрегации:

count()	          Total number of items
first(), last()	  First and last item
mean(), median()  Mean and median
min(), max()	  Minimum and maximum
std(), var()	  Standard deviation and variance
mad()	          Mean absolute deviation
prod()	          Product of all items
sum()	          Sum of all items
size()            Number of rows in each group

In [4]:
planets.groupby('method')
# groupby('столбец фрейма') - разбивает и группирует фрейм по элементам 
# выбранного столбца, создавая новый объект:
# pandas.core.groupby.generic.DataFrameGroupBy, с которым можно проводить
# вычисления min, max, mean, etc.

planets.groupby('method')['orbital_period'].median()
# группировка данных по method и вычисление медианы для каждого method
# по значению orbital_period

#for (method, group) in planets.groupby('method'):
#    print("{0:30s} shape={1}".format(method, group.shape))
# groupby поддерживает иттерации по циклам, но забей на это, ирл редко юзается

planets.groupby('method')['year'].describe()
# describe() можно использовать вместе с groupby() для получения описания
# агрегированного фрейма.
# В этом примере мы смотрим на описание year по method, это позволяет получить
# много информации о том какие методы самые частые, редкие, в каком году они
# начали использоваться и т.д.



,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


In [5]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])

df.groupby('key').aggregate(['min', np.median, max])
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})
# groupby().aggregate() может принимать строку, функцию, список или словарь
# для агрегации фрейма, что позволяет задать КАК мы хотим провести агрегацию

def filter_func(x):
    return x['data2'].std() > 4 #std - вычисление standard deviation

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")
# .groupby().filter() - позволяет отбрасывать данные в соответствии с указаной
# фильтрующей функией (которую мы задаем), котороя должна возращать булево
# значение, показывающее проходит ли указанная группа данных через фильтр



df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

In [6]:
df.groupby('key').transform(lambda x: x - x.mean())
# groupby().transform() возвращает трансформированную версию ВСЕГО фрейма
# в то время как обычная агрегация возвращает "сжатую/уменьшенную" версию

def norm_by_data2(x):
    # x - DataFrame of group values
    x['data1'] /= x['data2'].sum() # x /= a то же самое что x = x / a
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")
# .gropby().apply() позволяет применить ЛЮБУЮ функцию к результатам агрегации
# функция должна принимать DataFrame и возвращать скаляр или объект pandas

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


In [7]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')
# "ключём" для .groupby() может быть и список, и массив, и Series,
# при условии что его длинна равна таковой у DataFrame который мы агрегируем

df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')
# еще в качестве ключа можно использовать словарь связывающий индексы с ключами
# групп

display('df2', 'df2.groupby(str.lower).mean()')
# можно передать как ключ любую функцию принимающую значения индексов
# и возвращающую группу

df2.groupby([str.lower, mapping]).mean()
# все вышеуказанные способы задания ключей можно использовать для мульти-индекса



,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


In [13]:
# метод .agg() позволяет провести сразу несколько операций с аггрегированными данными

planets.groupby('method')['orbital_period'].agg([min, max])
# еще можно писать как .agg(['min', 'max'])

# sort_values(by=['min', 'max'], ascending=False) - сортировка по значениям столбцов
# by=['min', 'max'] не работает при использовании с .groupby()


,min,max
method,,
Astrometry,246.360000,1016.000000
Eclipse Timing Variations,1916.250000,10220.000000
Imaging,4639.150000,730000.000000
Microlensing,1825.000000,5100.000000
Orbital Brightness Modulation,0.240104,1.544929
Pulsar Timing,0.090706,36525.000000
Pulsation Timing Variations,1170.000000,1170.000000
Radial Velocity,0.736540,17337.500000
Transit,0.355000,331.600590
